In [1]:
### Library Import Initialization
import numpy as np
import pandas as pd
import yfinance as yf
from scipy.stats import norm


In [8]:
### Function to Commodities Data
# Define the list of commodity tickers
#tickers = ['GC=F',  # Gold futures
#           'SI=F',  # Silver futures
#           'CL=F',  # Crude oil futures
#           'HG=F',  # Copper futures
#           'NG=F']  # Natural gas futures
tickers = ['HG=F', 'SI=F', 'CL=F']

def import_commod_data(tickers, start_date):
    data = pd.DataFrame()
    if len(tickers) == 1:
        data[tickers[0]] = yf.download(tickers[0], start_date)['Adj Close']
    else:
        for t in tickers:
            data[t] = yf.download(t, start_date)['Adj Close']
    return data

start_date = '2022-01-01'
commod_data = import_commod_data(tickers, start_date)
commod_data = commod_data.reset_index() # Turn Multi-Index into Column in Pandas 
commod_data.tail()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


,Date,HG=F,SI=F,CL=F
635,2024-07-15,4.529,30.672001,81.910004
636,2024-07-16,4.444,31.195000,80.760002
637,2024-07-17,4.399,30.136999,82.849998
638,2024-07-18,4.266,29.995001,82.820000
639,2024-07-19,4.219,29.087999,80.129997


In [3]:
### Compute Daily Returns
def daily_returns(commod_data):
    # Drop the date column
    commod_data = commod_data.drop(columns = ['Date'])
    # Compute the percentage change
    rets = commod_data.pct_change().dropna()
    
    return rets

returns = daily_returns(commod_data)
print(returns.tail())


         HG=F      SI=F      CL=F
635 -0.015114 -0.006929 -0.003649
636 -0.018768  0.017051 -0.014040
637 -0.010126 -0.033916  0.025879
638 -0.030234 -0.004712 -0.000362
639 -0.011017 -0.030238 -0.032480


In [4]:
### Function to compute sigma (std dev of log normal historical stock return data)
def compute_sigma(rets):
    # Compute the standard deviation of returns
    sigma = np.std(rets) * np.sqrt(252) # annualize the standard deviation

    return sigma

sigma = compute_sigma(returns)
print(sigma)


HG=F    0.241712
SI=F    0.295619
CL=F    0.384252
dtype: float64


In [5]:
### Function to compute Black76 Model
def Black76(option_type, r, S_0, K, T, sigma):
    # Convert S_0 and K to NumPy arrays for element-wise operations
    S_0 = np.array(S_0)
    K = np.array(K)

    # Calculate d_1 and d_2 element-wise
    d_1 = (np.log(S_0 / K) + 0.5 * sigma**2 * T) / (sigma * np.sqrt(T))
    d_2 = (np.log(S_0 / K) - 0.5 * sigma**2 * T) / (sigma * np.sqrt(T))

    if option_type == 'call':
        option_value = (np.exp(-r * T)) * (S_0 * norm.cdf(d_1) - K * norm.cdf(d_2))
    elif option_type == 'put':
        option_value = (np.exp(-r * T)) * (K * norm.cdf(-d_2) - S_0 * norm.cdf(-d_1))

    return option_value
    

In [6]:
### Return Black76 Option Values
# Extract most recent commodity prices as a list
S_0 = commod_data.iloc[-1, 1:].values.tolist()
# Create Strike Price list
K = [6, 28.5, 79.5]
# Type Specification and other metrics
option_type = 'put'
r = 0.05
T = 1

# Call B76 Function
option_val = Black76(option_type, r, S_0, K, T, sigma)
#print(option_val)
# Return results in a printed list format
option_val_list = []
for i, ticker in enumerate(tickers):
    option_val_list.append((ticker, option_val[i]))

# Print each item in output on a new line
for item in option_val_list:
    print(f"{item[0]}: {round(item[1], 4)}")


HG=F: 1.7313
SI=F: 2.9464
CL=F: 11.27


In [7]:
### Manual Test
S_0 = 42
K = 42
T = 1.5
r = 0.05
sigma = 0.20
option_type = 'call'

test = Black76(option_type, r, S_0, K, T, sigma)
print(f'The value of the put given the above inputs is: ${round(test, 4)}\nand the answer is supposed to be: $3.7982')


The value of the put given the above inputs is: $3.7982
and the answer is supposed to be: $3.7982
